In [3]:
# Import our dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# Load the charity_data.csv into a Pandas DataFrame
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

# Display the first few rows to confirm the data loaded correctly
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
# These columns are not useful for predicting the target variable and should be removed.
application_df.drop(columns=['EIN', 'NAME'], inplace=True)

# Display the first few rows to confirm the columns were dropped
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Determine the number of unique values in each column.
# This helps us understand the distribution and variety of values in each feature,
# which is important for optimizing the model.
unique_values = application_df.nunique()

# Display the number of unique values for each column
unique_values

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
# Grouping rare application types to reduce the number of unique values and simplify the model.
app_type_counts = application_df["APPLICATION_TYPE"].value_counts()

# Choose a cutoff value and create a list of application types to be replaced
# Use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type_counts[app_type_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
# Grouping rare classifications to reduce the number of unique values and simplify the model.
classification_counts = application_df["CLASSIFICATION"].value_counts()

# Choose a cutoff value and create a list of classifications to be replaced
# Use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
# This step converts all remaining categorical columns into numeric format,
# creating one-hot encoded binary columns for each unique value.
application_df = pd.get_dummies(application_df)

# Display the first few rows to confirm the encoding
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False


In [9]:
# Split our preprocessed data into our features and target arrays
# The target array (y) is the 'IS_SUCCESSFUL' column, which we want to predict.
# The features array (X) includes all remaining columns.
X = application_df.drop(columns=['IS_SUCCESSFUL'])
y = application_df['IS_SUCCESSFUL']

# Display the shape of the features and target arrays to confirm the split
X.shape, y.shape

((34299, 43), (34299,))

In [10]:
# Split the preprocessed data into a training and testing dataset
# We use an 80-20 split to train the model on the majority of the data while reserving a portion for testing.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Display the shapes of the resulting datasets to confirm the split
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((27439, 43), (6860, 43), (27439,), (6860,))

In [11]:
# Create a StandardScaler instance
# Scaling the features ensures that all input values are on a similar scale, which helps the neural network learn effectively.
scaler = StandardScaler()

# Fit the StandardScaler to the training data to calculate the scaling parameters
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Display the first few rows of the scaled training data to confirm the transformation
X_train_scaled[:5]

array([[ 0.01045683, -0.03095675, -0.08845001, -0.1248287 , -0.17922996,
         0.5200401 , -0.2177384 , -0.18917119, -0.192759  , -0.14643132,
        -0.15002639, -0.92621703, -0.0431524 ,  0.93255087, -0.03196072,
        -0.01045683, -0.02002623,  0.99371486, -0.40606734, -0.46568343,
        -0.24106748, -0.2430079 , -0.26669204, -0.10549274, -0.0662763 ,
        -0.01045683,  0.47129428, -0.44655824, -0.65552292, -0.11802954,
        -0.03573774,  0.67902549,  0.63716135, -0.14989936, -0.12452723,
        -0.33110845, -0.08238929, -0.16855186, -0.34918231, -0.06401961,
        -0.07488171,  0.02832705, -0.02832705],
       [ 0.01045683, -0.03095675, -0.08845001, -0.1248287 , -0.17922996,
         0.5200401 , -0.2177384 , -0.18917119, -0.192759  , -0.14643132,
        -0.15002639, -0.92621703, -0.0431524 ,  0.93255087, -0.03196072,
        -0.01045683, -0.02002623, -1.0063249 , -0.40606734,  2.14738153,
        -0.24106748, -0.2430079 , -0.26669204, -0.10549274, -0.0662763 ,
   

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# We will start with a slightly larger model to attempt to push accuracy above 75%.
number_input_features = len(X_train_scaled[0])

# Create the Sequential model
optimized_nn = tf.keras.models.Sequential()

# First hidden layer
# Increasing the number of nodes to 100 to capture more complex relationships.
optimized_nn.add(tf.keras.layers.Dense(units=100, input_dim=number_input_features, activation="relu"))

# Second hidden layer
# Adding a second hidden layer with 50 nodes for additional learning capacity.
optimized_nn.add(tf.keras.layers.Dense(units=50, activation="relu"))

# Third hidden layer (optimization attempt #1)
# Adding a third hidden layer with 25 nodes for deeper learning.
optimized_nn.add(tf.keras.layers.Dense(units=25, activation="relu"))

# Output layer
# Using the sigmoid activation function for binary classification.
optimized_nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the optimized model
optimized_nn.summary()

/opt/anaconda3/envs/dev/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 100)            │         4,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 25)             │         1,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,751 (42.00 KB)

 Trainable params: 10,751 (42.00 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Compile the optimized model
# Using the 'adam' optimizer for adaptive learning rates and 'binary_crossentropy' for binary classification.
optimized_nn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [15]:
# Define the callback to save the model's weights every 5 epochs
# This ensures we capture the best version of the model at regular intervals.
checkpoint_path = "optimized_checkpoints/weights.{epoch:02d}.weights.h5"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, save_freq=5*len(X_train_scaled))

# Train the optimized model
# Using 100 epochs to match the sample output and give the model enough time to learn.
optimized_history = optimized_nn.fit(X_train_scaled, y_train, epochs=100, batch_size=32, callbacks=[checkpoint_callback])

Epoch 1/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 327us/step - accuracy: 0.7112 - loss: 0.5846
Epoch 2/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 330us/step - accuracy: 0.7302 - loss: 0.5536
Epoch 3/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 319us/step - accuracy: 0.7326 - loss: 0.5481
Epoch 4/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 320us/step - accuracy: 0.7316 - loss: 0.5521
Epoch 5/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 313us/step - accuracy: 0.7306 - loss: 0.5509
Epoch 6/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 324us/step - accuracy: 0.7335 - loss: 0.5470
Epoch 7/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 313us/step - accuracy: 0.7324 - loss: 0.5478
Epoch 8/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 311us/step - accuracy: 0.7391 - loss: 0.5389
Epoch 9/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 315us/step - accuracy: 0.7310 - loss: 0.5479
Epoch 10/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 328us/step - accuracy: 0.7356 - loss: 0.5440
Epoch 11/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 315us/step - accuracy: 0.7368 - loss: 0.5414
Epoch 12/100
858/85

In [16]:
# Evaluate the optimized model using the test data
optimized_model_loss, optimized_model_accuracy = optimized_nn.evaluate(X_test_scaled, y_test, verbose=2)

# Print the results to confirm the model's performance
print(f"Optimized Loss: {optimized_model_loss}, Optimized Accuracy: {optimized_model_accuracy}")

215/215 - 0s - 477us/step - accuracy: 0.7276 - loss: 0.5750
Optimized Loss: 0.5750491619110107, Optimized Accuracy: 0.7275510430335999


In [17]:
# Define a new optimized model with more neurons (Optimization Attempt #2)
number_input_features = len(X_train_scaled[0])

# Create the Sequential model
optimized_nn_v2 = tf.keras.models.Sequential()

# First hidden layer - increased to 128 neurons
optimized_nn_v2.add(tf.keras.layers.Dense(units=128, input_dim=number_input_features, activation="relu"))

# Second hidden layer - increased to 64 neurons
optimized_nn_v2.add(tf.keras.layers.Dense(units=64, activation="relu"))

# Third hidden layer - increased to 32 neurons
optimized_nn_v2.add(tf.keras.layers.Dense(units=32, activation="relu"))

# Output layer
optimized_nn_v2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
optimized_nn_v2.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model with the same 100 epochs for consistency
optimized_history_v2 = optimized_nn_v2.fit(X_train_scaled, y_train, epochs=100, batch_size=32, verbose=1)

Epoch 1/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 352us/step - accuracy: 0.7130 - loss: 0.5807
Epoch 2/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 340us/step - accuracy: 0.7334 - loss: 0.5500
Epoch 3/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 330us/step - accuracy: 0.7263 - loss: 0.5545
Epoch 4/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 344us/step - accuracy: 0.7339 - loss: 0.5496
Epoch 5/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 333us/step - accuracy: 0.7322 - loss: 0.5454
Epoch 6/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 337us/step - accuracy: 0.7378 - loss: 0.5403
Epoch 7/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 332us/step - accuracy: 0.7319 - loss: 0.5470
Epoch 8/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 349us/step - accuracy: 0.7350 - loss: 0.5426
Epoch 9/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 330us/step - accuracy: 0.7338 - loss: 0.5437
Epoch 10/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 335us/step - accuracy: 0.7406 - loss: 0.5391
Epoch 11/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 330us/step - accuracy: 0.7356 - loss: 0.5454
Epoch 12/100
858/85

In [18]:
# Evaluate the second optimized model using the test data
optimized_model_v2_loss, optimized_model_v2_accuracy = optimized_nn_v2.evaluate(X_test_scaled, y_test, verbose=2)

# Print the results to confirm the model's performance
print(f"Optimized V2 Loss: {optimized_model_v2_loss}, Optimized V2 Accuracy: {optimized_model_v2_accuracy}")

215/215 - 0s - 494us/step - accuracy: 0.7264 - loss: 0.5685
Optimized V2 Loss: 0.5684700608253479, Optimized V2 Accuracy: 0.7263848185539246


In [19]:
# Define a new optimized model with dropout layers (Optimization Attempt #3)
number_input_features = len(X_train_scaled[0])

# Create the Sequential model
optimized_nn_v3 = tf.keras.models.Sequential()

# First hidden layer with dropout
optimized_nn_v3.add(tf.keras.layers.Dense(units=128, input_dim=number_input_features, activation="relu"))
optimized_nn_v3.add(tf.keras.layers.Dropout(0.2))  # Drop 20% of the neurons

# Second hidden layer with dropout
optimized_nn_v3.add(tf.keras.layers.Dense(units=64, activation="relu"))
optimized_nn_v3.add(tf.keras.layers.Dropout(0.2))  # Drop 20% of the neurons

# Third hidden layer with dropout
optimized_nn_v3.add(tf.keras.layers.Dense(units=32, activation="relu"))
optimized_nn_v3.add(tf.keras.layers.Dropout(0.2))  # Drop 20% of the neurons

# Output layer
optimized_nn_v3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
optimized_nn_v3.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model with the same 100 epochs for consistency
optimized_history_v3 = optimized_nn_v3.fit(X_train_scaled, y_train, epochs=100, batch_size=32, verbose=1)

Epoch 1/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 379us/step - accuracy: 0.6889 - loss: 0.6056
Epoch 2/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 364us/step - accuracy: 0.7244 - loss: 0.5696
Epoch 3/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 354us/step - accuracy: 0.7261 - loss: 0.5606
Epoch 4/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 359us/step - accuracy: 0.7289 - loss: 0.5579
Epoch 5/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 363us/step - accuracy: 0.7298 - loss: 0.5565
Epoch 6/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 380us/step - accuracy: 0.7302 - loss: 0.5590
Epoch 7/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 353us/step - accuracy: 0.7277 - loss: 0.5557
Epoch 8/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 354us/step - accuracy: 0.7244 - loss: 0.5624
Epoch 9/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 353us/step - accuracy: 0.7303 - loss: 0.5520
Epoch 10/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 353us/step - accuracy: 0.7292 - loss: 0.5535
Epoch 11/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 353us/step - accuracy: 0.7318 - loss: 0.5504
Epoch 12/100
858/85

In [20]:
# Evaluate the third optimized model using the test data
optimized_model_v3_loss, optimized_model_v3_accuracy = optimized_nn_v3.evaluate(X_test_scaled, y_test, verbose=2)

# Print the results to confirm the model's performance
print(f"Optimized V3 Loss: {optimized_model_v3_loss}, Optimized V3 Accuracy: {optimized_model_v3_accuracy}")

215/215 - 0s - 497us/step - accuracy: 0.7290 - loss: 0.5561
Optimized V3 Loss: 0.5561376214027405, Optimized V3 Accuracy: 0.7290087342262268
